In [3]:
import folium
import pandas as pd

import json

# load the dataset
df = pd.read_csv('kc_house_data.csv')

# calculate median price per zipcode
zipcode_median = df.groupby(['zipcode']).median()

# df transformations
zipcode_median.reset_index(level=0, inplace=True)
zipcode_median.zipcode = zipcode_median.zipcode.astype(str)

# load the Washington state zipcodes FeatureCollection
path='./wa_washington_zip_codes_geo.min.json'
with open(path) as json_file:  
    WA_zip = json.load(json_file)

zipcode_list = list(zipcode_median.zipcode.unique())
WA_zip['features'] = [zipcode for zipcode in WA_zip['features'] if zipcode['properties']['ZCTA5CE10'] in zipcode_list]

In [4]:
m = folium.Map(location=[47.4, -122], zoom_start=9)

folium.Choropleth(highlight=True,
    geo_data=WA_zip,
    name='choropleth',
    data=zipcode_median,
    columns=['zipcode', 'price'],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='house prices (median)'
).add_to(m)

folium.LayerControl().add_to(m)

m